In [13]:
from glob import glob
import pandas as pd
from IPython.display import display
from tqdm import tqdm
blast_directory = "/homes/madelarambelje/blast_output/"
filenames = sorted(glob("/home/madelarambelje/complete_blast/*.out"))
colnames = ["Read ID", "Scientific Name", "Tax ID", "e-value","bitscore", "length", "qcov" ]
blast_files = [pd.read_csv(file, sep = "\t", header=None, names=colnames) for file in filenames]

In [88]:
!pip3 install tqdm

    100% |████████████████████████████████| 81kB 2.4MB/s ta 0:00:011


In [14]:
def merge_blast_files(blast_files, output_file):
    '''Merges the input in to one dataframe, with subject labels, 
an extra column is_vegan which holds a boolean. 
This function also filters out the lowest e-value per read. '''
    dfs = []
    subjects = ["A","B","C","D","E"]

    for i, blast_file in tqdm(enumerate(blast_files[:5])):
        blast_file["Read ID"] = blast_file["Read ID"].str.strip("_read_number").astype(int)
        blast_file = blast_file.loc[blast_file.groupby("Read ID")["e-value"].idxmin()]
        blast_file.loc[:,"subject"] = subjects[i]
        blast_file.loc[:,"is_vegan"] = True
        dfs.append(blast_file)
    for i, blast_file in tqdm(enumerate(blast_files[5:])):
        blast_file["Read ID"] = blast_file["Read ID"].str.strip("_read_number").astype(int)
        blast_file = blast_file.loc[blast_file.groupby("Read ID")["e-value"].idxmin()]
        blast_file.loc[:,"subject"] = subjects[i]
        blast_file.loc[:,"is_vegan"] = False
        dfs.append(blast_file)
    merged_blast = pd.concat(dfs)
    merged_blast = merged_blast.set_index(['subject', "is_vegan"])
    merged_blast = merged_blast[merged_blast['e-value']<10e-10]
    merged_blast.to_csv(output_file, sep="\t")
    
    return merged_blast

merged_blast = merge_blast_files(blast_files=blast_files, output_file="merged_blast.txt")


5it [00:52, 10.47s/it]
5it [00:56, 11.24s/it]


In [33]:

#len(merged_blast[merged_blast['e-value']<0.001])

# def count_reads_per_subject(is_vegan):
#     subjects = ["A","B","C","D","E"]
#     if is_vegan == True:
#         print("Vegan")
#     else:
#         print("Control")
#     for subject in subjects:
#         print("Reads subject",subject,":",len(merged_blast[merged_blast["is_vegan"]==is_vegan].loc[subject]))
# count_reads_per_subject(is_vegan=True)
# count_reads_per_subject(is_vegan=False)



Read ID                                    Scientific Name  \
subject is_vegan                                                               
A       True            1                                   Prevotella copri   
        True            2                              Bacteroides uniformis   
        True            3                                uncultured organism   
        True            4                               Alistipes sp. 5CBH24   
        True            6                    Lachnospiraceae bacterium GAM79   
...                   ...                                                ...   
E       True        12126                               Dialister sp. 5BBH33   
        True        12127                                   Escherichia coli   
        True        12128                               uncultured bacterium   
        True        12129                 Cloning vector pAH-mini-Mu(LER)-YS   
        True        12130  [Eubacterium] rectale ATCC 33656;Lachnospirace...   

                          Tax ID        e-value  bitscore  length  qcov  
subject is_vegan                                                         
A       True              165179  6.610000e-157     566.0     449    76  
        True                 820  3.620000e-128     470.0     292    72  
        True              155900   0.000000e+00     678.0     458    14  
        True             2585118   0.000000e+00     900.0     716    35  
        True             2109691   5.710000e-29     139.0     104    47  
...                          ...            ...       ...     ...   ...  
E       True             2582419   2.520000e-16      97.1      52    32  
        True                 562   7.550000e-10      78.7      54     5  
        True               77133   1.960000e-11      80.5      60    43  
        True             2049510   1.220000e-13      89.8      55    13  
        True      515619;1898203   0.000000e+00     928.0     845    88  

[387714 rows x 7 columns]